In [1]:
from rdkit import Chem
import pandas as pd
import numpy as np

from importlib import reload

import rcm
file_path_connectivity_table = 'cp6t6_0p_blyp35_geom_connectivity_global.csv'
xyz_file_path = 'cp6t6_0p_blyp35_geom.xyz'

RCM module loaded.


In [4]:
a = rcm.RCM(xyz_file_path,file_path_connectivity_table)

In [9]:
a.get_B(0,0,0)

Bx   -0.000013
By    0.000010
Bz   -0.037761
dtype: float64